In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1"

In [2]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import matrix_product_state
from tensorly import tt_to_tensor
from scipy.sparse import csr_matrix
import torch
from multiprocessing import Pool
import scipy
from tensorly.tenalg import contract
from tensorly.tucker_tensor import tucker_to_tensor
from tensorly.cp_tensor import cp_to_tensor
from tensorly.tenalg import multi_mode_dot
from tensorly.decomposition import tucker

In [3]:
def tensordot(A,B,modes):
    return contract(A,modes[0],B,modes[1])

In [4]:
def TTOI(Y_tensor, r_vec, iter, tol):
    dim_vec = Y_tensor.shape
    d = len(dim_vec)
    X_hat_arr = [None] * iter
    V_prod_arr = [None] * int(np.floor(iter/2)+1)
    U_prod_arr = [None] * int(np.ceil(iter/2))
    Y_arr = [None] * (d-1)
    for i in range(d-1):
        Y_arr[i] = Y_tensor.reshape(np.prod(dim_vec[:i+1]), np.prod(dim_vec[i+1:]), order="F")
    V_prod_arr[0] = [1] * (d-1)
    n = 0
    chg = np.Inf
    while n<iter and chg > tol:
        if n==0:
            U_prod_arr[int(n / 2)] = [None] * (d - 1)
            Y_tilde_arr = [None] * (d - 2)
            U_temp, _, _ = torch.svd(torch.tensor(Y_arr[0]))
            U_temp = U_temp.numpy()
            U_temp = U_temp[:, :r_vec[0]]
            U_prod_arr[int(n / 2)][0] = U_temp
            for k in range(1, d - 1):
                Y_temp = np.kron(np.eye(dim_vec[k]), U_prod_arr[int(n / 2)][k - 1]).T @ Y_arr[k]
                Y_tilde_arr[k - 1] = Y_temp.reshape(r_vec[k - 1], np.prod(dim_vec[k:]), order="F")
                U_temp, _, _ = np.linalg.svd(Y_temp)
                U_temp = U_temp[:, :r_vec[k]]
                U_prod_arr[int(n / 2)][k] = np.kron(np.eye(dim_vec[k]), U_prod_arr[int(n / 2)][k - 1]) @ U_temp
            X_hat_temp = U_prod_arr[int(n / 2)][d - 2].T @ Y_arr[d - 2]
            X_hat_arr[n] = (U_prod_arr[int(n / 2)][d - 2] @ X_hat_temp).reshape(dim_vec, order="F")
        elif n % 2 == 0:
            U_prod_arr[int(n/2)] = [None] * (d-1)
            Y_tilde_arr = [None] * (d-2)
            U_temp, _, _ = np.linalg.svd(Y_arr[0]@V_prod_arr[int(n/2)][d-2])
            U_temp = U_temp[:,:r_vec[0]]
            U_prod_arr[int(n/2)][0] = U_temp
            for k in range(1,d-1):
                Y_temp = np.kron(np.eye(dim_vec[k]),U_prod_arr[int(n/2)][k-1]).T@Y_arr[k]
                Y_tilde_arr[k-1] = Y_temp.reshape(r_vec[k-1],np.prod(dim_vec[k:]),order="F")
                U_temp, _, _ = np.linalg.svd(Y_temp@V_prod_arr[int(n/2)][d-k-2])
                U_temp = U_temp[:,:r_vec[k]]
                U_prod_arr[int(n/2)][k] = np.kron(np.eye(dim_vec[k]),U_prod_arr[int(n/2)][k-1])@U_temp
            X_hat_temp = U_prod_arr[int(n/2)][d-2].T@Y_arr[d-2]
            X_hat_arr[n] = (U_prod_arr[int(n/2)][d-2]@X_hat_temp).reshape(dim_vec,order="F")
        else:
            V_prod_arr[int((n+1)/2)] = [None] * (d-1)
            _, _, V_temp = np.linalg.svd(U_prod_arr[int((n-1)/2)][d-2].T@Y_arr[d-2])
            V_temp = V_temp[:r_vec[d-2],:].T
            V_prod_arr[int((n+1)/2)][0] = V_temp
            for k in range(1,d-1):
                _, _, V_temp = np.linalg.svd(Y_tilde_arr[d-k-2]@np.kron(V_prod_arr[int((n+1)/2)][k-1],np.eye(dim_vec[d-k-1])))
                V_temp = V_temp[:r_vec[d - k-2], :].T
                V_prod_arr[int((n+1)/2)][k] = np.kron(V_prod_arr[int((n+1)/2)][k-1],np.eye(dim_vec[d-k-1]))@V_temp
            X_hat_temp = Y_arr[0]@V_prod_arr[int((n+1)/2)][d-2]
            X_hat_arr[n] = (X_hat_temp@V_prod_arr[int((n+1)/2)][d-2].T).reshape(dim_vec,order="F")
        n=n+1
        if n>1:
            chg = np.square(np.linalg.norm(X_hat_arr[n-1].reshape(np.prod(dim_vec),1),"fro"))-np.square(np.linalg.norm(X_hat_arr[n-2].reshape(np.prod(dim_vec),1),"fro"))

    return [x for x in X_hat_arr if x is not None]

In [5]:
def RGrad3(X, ranks, A):
    p1, p2, p3 = X.shape
    _, r1, r2,  _ = ranks
    factors = matrix_product_state(X, rank=ranks)
    G1, G2, G3 = factors
    G1 = G1.reshape(p1,r1,order="F")
    G3 = G3.reshape(r2,p3,order="F")

    G_ge2 = (tensordot(G2,G3,[2,0])).reshape(r1, p2*p3, order="F")
    A1 = (np.identity(p1)-G1@G1.T)@A.reshape(p1,p2*p3,order="F")@G_ge2.T@np.linalg.inv(G_ge2@G_ge2.T)
    delta_A1 = tensordot(tensordot(A1,G2,[1,0]),G3,[2,0])

    L_G2 = G2.reshape(p2*r1, r2, order="F")
    A2 = (np.identity(p2*r1)-L_G2@L_G2.T)@(np.kron(np.identity(p2), G1)).T@A.reshape(p1*p2,p3,order="F")@G3.T@np.linalg.inv(G3@G3.T)
    delta_A2 = tensordot(tensordot(G1, A2.reshape([r1,p2,r2],order="F"), [1, 0]), G3, [2, 0])

    G_le2 = (tensordot(G1,G2,[1,0])).reshape(p1*p2,r2,order="F")
    A3 = (np.kron(np.identity(p3), G_le2)).T@A.reshape(p1*p2*p3,1,order="F")
    delta_A3 = tensordot(tensordot(G1, G2, [1, 0]), A3.reshape(r2,p3,order="F"), [2, 0])

    return delta_A1+delta_A2+delta_A3

In [6]:
def RGrad4(X, ranks, A):
    p1, p2, p3, p4 = X.shape
    _, r1, r2, r3, _ = ranks
    factors = matrix_product_state(X, rank=ranks)
    G1, G2, G3, G4 = factors
    G1 = G1.reshape(p1,r1,order="F")
    G4 = G4.reshape(r3,p4,order="F")

    G_ge2 = (tensordot(tensordot(G2,G3,[2,0]),G4,[3,0])).reshape(r1, p2*p3*p4, order="F")
    A1 = (np.identity(p1)-G1@G1.T)@A.reshape(p1,p2*p3*p4,order="F")@G_ge2.T@np.linalg.inv(G_ge2@G_ge2.T)
    delta_A1 = tensordot(tensordot(tensordot(A1,G2,[1,0]),G3,[2,0]),G4,[3,0])

    G_ge3 = (tensordot(G3,G4,[2,0])).reshape(r2, p3*p4, order="F")
    L_G2 = G2.reshape(p2*r1, r2, order="F")
    A2 = (np.identity(p2*r1)-L_G2@L_G2.T)@(np.kron(np.identity(p2), G1)).T@A.reshape(p1*p2,p3*p4,order="F")@G_ge3.T@np.linalg.inv(G_ge3@G_ge3.T)
    delta_A2 = tensordot(tensordot(tensordot(G1, A2.reshape([r1,p2,r2],order="F"), [1, 0]), G3, [2, 0]), G4, [3, 0])

    L_G3 = G3.reshape(p3*r2, r3, order="F")
    G_le2 = (tensordot(G1,G2,[1,0])).reshape(p1*p2,r2, order="F")
    A3 = (np.identity(p3*r2)-L_G3@L_G3.T)@(np.kron(np.identity(p3), G_le2)).T@A.reshape(p1*p2*p3,p4,order="F")@G4.T@np.linalg.inv(G4@G4.T)
    delta_A3 = tensordot(tensordot(tensordot(G1, G2, [1, 0]), A3.reshape([r2,p3,r3],order="F"), [2, 0]), G4, [3, 0])

    G_le3 = (tensordot(tensordot(G1,G2,[1,0]),G3,[2,0])).reshape(p1*p2*p3,r3,order="F")
    A4 = (np.kron(np.identity(p4), G_le3)).T@A.reshape(p1*p2*p3*p4,1,order="F")
    delta_A4 = tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), A4.reshape(r3,p4,order="F"), [3, 0])

    return delta_A1+delta_A2+delta_A3+delta_A4

In [7]:
def RGrad5(X, ranks, A):
    p1, p2, p3, p4, p5 = X.shape
    _, r1, r2, r3, r4, _ = ranks
    factors = matrix_product_state(X, rank=ranks)
    G1, G2, G3, G4, G5 = factors
    G1 = G1.reshape(p1,r1,order="F")
    G5 = G5.reshape(r4,p5,order="F")

    G_ge2 = (tensordot(tensordot(tensordot(G2,G3,[2,0]),G4,[3,0]),G5,[4,0])).reshape(r1, p2*p3*p4*p5, order="F")
    A1 = (np.identity(p1)-G1@G1.T)@A.reshape(p1,p2*p3*p4*p5,order="F")@G_ge2.T@np.linalg.inv(G_ge2@G_ge2.T)
    delta_A1 = tensordot(tensordot(tensordot(tensordot(A1,G2,[1,0]),G3,[2,0]),G4,[3,0]),G5,[4,0])

    G_ge3 = (tensordot(tensordot(G3,G4,[2,0]),G5,[3,0])).reshape(r2, p3*p4*p5, order="F")
    L_G2 = G2.reshape(p2*r1, r2, order="F")
    A2 = (np.identity(p2*r1)-L_G2@L_G2.T)@(np.kron(np.identity(p2), G1)).T@A.reshape(p1*p2,p3*p4*p5,order="F")@G_ge3.T@np.linalg.inv(G_ge3@G_ge3.T)
    delta_A2 = tensordot(tensordot(tensordot(tensordot(G1, A2.reshape([r1,p2,r2],order="F"), [1, 0]), G3, [2, 0]), G4, [3, 0]), G5, [4,0])

    G_ge4 = (tensordot(G4,G5,[2,0])).reshape(r3, p4*p5, order="F")
    L_G3 = G3.reshape(p3*r2, r3, order="F")
    G_le2 = (tensordot(G1,G2,[1,0])).reshape(p1*p2,r2, order="F")
    A3 = (np.identity(p3*r2)-L_G3@L_G3.T)@(np.kron(np.identity(p3), G_le2)).T@A.reshape(p1*p2*p3,p4*p5,order="F")@G_ge4.T@np.linalg.inv(G_ge4@G_ge4.T)
    delta_A3 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), A3.reshape([r2,p3,r3],order="F"), [2, 0]), G4, [3, 0]), G5, [4,0])

    L_G4 = G4.reshape(p4*r3, r4, order="F")
    G_le3 = (tensordot(tensordot(G1,G2,[1,0]),G3,[2,0])).reshape(p1*p2*p3,r3, order="F")
    A4 = (np.identity(p4*r3)-L_G4@L_G4.T)@(np.kron(np.identity(p4),G_le3)).T@A.reshape(p1*p2*p3*p4,p5,order="F")@G5.T @np.linalg.inv(G5@G5.T)
    delta_A4 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), A4.reshape([r3,p4,r4], order="F"), [3, 0]),G5,[4,0])

    G_le4 = (tensordot(tensordot(tensordot(G1,G2,[1,0]),G3,[2,0]),G4,[3,0])).reshape(p1*p2*p3*p4,r4,order="F")
    A5 = (np.kron(np.identity(p5), G_le4)).T@A.reshape(p1*p2*p3*p4*p5,1,order="F")
    delta_A5 = tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), G4,[3,0]),A5.reshape(r4,p5,order="F"), [4, 0])

    return delta_A1+delta_A2+delta_A3+delta_A4+delta_A5

In [8]:
def RGrad6(X, ranks, A):
    p1, p2, p3, p4, p5, p6 = X.shape
    _, r1, r2, r3, r4, r5, _ = ranks
    factors = matrix_product_state(X, rank=ranks)
    G1, G2, G3, G4, G5, G6 = factors
    G1 = G1.reshape(p1,r1,order="F")
    G6 = G6.reshape(r5,p6,order="F")

    G_ge2 = (tensordot(tensordot(tensordot(tensordot(G2,G3,[2,0]),G4,[3,0]),G5,[4,0]),G6,[5,0])).reshape(r1, p2*p3*p4*p5*p6, order="F")
    A1 = (np.identity(p1)-G1@G1.T)@A.reshape(p1,p2*p3*p4*p5*p6,order="F")@G_ge2.T@np.linalg.inv(G_ge2@G_ge2.T)
    delta_A1 = tensordot(tensordot(tensordot(tensordot(tensordot(A1,G2,[1,0]),G3,[2,0]),G4,[3,0]),G5,[4,0]),G6,[5,0])

    G_ge3 = (tensordot(tensordot(tensordot(G3,G4,[2,0]),G5,[3,0]),G6,[4,0])).reshape(r2, p3*p4*p5*p6, order="F")
    L_G2 = G2.reshape(p2*r1, r2, order="F")
    A2 = (np.identity(p2*r1)-L_G2@L_G2.T)@(np.kron(np.identity(p2), G1)).T@A.reshape(p1*p2,p3*p4*p5*p6,order="F")@G_ge3.T@np.linalg.inv(G_ge3@G_ge3.T)
    delta_A2 = tensordot(tensordot(tensordot(tensordot(tensordot(G1, A2.reshape([r1,p2,r2],order="F"), [1, 0]), G3, [2, 0]), G4, [3, 0]), G5, [4,0]),G6,[5,0])

    G_ge4 = (tensordot(tensordot(G4,G5,[2,0]),G6,[3,0])).reshape(r3, p4*p5*p6, order="F")
    L_G3 = G3.reshape(p3*r2, r3, order="F")
    G_le2 = (tensordot(G1,G2,[1,0])).reshape(p1*p2,r2, order="F")
    A3 = (np.identity(p3*r2)-L_G3@L_G3.T)@(np.kron(np.identity(p3), G_le2)).T@A.reshape(p1*p2*p3,p4*p5*p6,order="F")@G_ge4.T@np.linalg.inv(G_ge4@G_ge4.T)
    delta_A3 = tensordot(tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), A3.reshape([r2,p3,r3],order="F"), [2, 0]), G4, [3, 0]), G5, [4,0]),G6,[5,0])

    G_ge5 = (tensordot(G5,G6,[2,0])).reshape(r4,p5*p6,order="F")
    L_G4 = G4.reshape(p4*r3, r4, order="F")
    G_le3 = (tensordot(tensordot(G1,G2,[1,0]),G3,[2,0])).reshape(p1*p2*p3,r3, order="F")
    A4 = (np.identity(p4*r3)-L_G4@L_G4.T)@(np.kron(np.identity(p4),G_le3)).T@A.reshape(p1*p2*p3*p4,p5*p6,order="F")@G_ge5.T @np.linalg.inv(G_ge5@G_ge5.T)
    delta_A4 = tensordot(tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), A4.reshape([r3,p4,r4], order="F"), [3, 0]),G5,[4,0]),G6,[5,0])

    L_G5 = G5.reshape(p5*r4, r5, order="F")
    G_le4 = (tensordot(tensordot(tensordot(G1,G2,[1,0]),G3,[2,0]),G4,[3,0])).reshape(p1*p2*p3*p4,r4, order="F")
    A5 = (np.identity(p5*r4)-L_G5@L_G5.T)@(np.kron(np.identity(p5),G_le4)).T@A.reshape(p1*p2*p3*p4*p5,p6,order="F")@G6.T @np.linalg.inv(G6@G6.T)
    delta_A5 = tensordot(tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), G4, [3, 0]),A5.reshape([r4,p5,r5],order="F"),[4,0]),G6,[5,0])

    G_le5 = (tensordot(tensordot(tensordot(tensordot(G1,G2,[1,0]),G3,[2,0]),G4,[3,0]),G5,[4,0])).reshape(p1*p2*p3*p4*p5,r5,order="F")
    A6 = (np.kron(np.identity(p6), G_le5)).T@A.reshape(p1*p2*p3*p4*p5*p6,1,order="F")
    delta_A6 = tensordot(tensordot(tensordot(tensordot(tensordot(G1, G2, [1, 0]), G3, [2, 0]), G4,[3,0]),G5,[4,0]),A6.reshape(r5,p6,order="F"), [5, 0])

    return delta_A1+delta_A2+delta_A3+delta_A4+delta_A5+delta_A6

In [9]:
def RGrad(X, ranks, A):
    if len(X.shape)==3:
        return RGrad3(X, ranks, A)
    elif len(X.shape)==4:
        return RGrad4(X, ranks, A)
    elif len(X.shape)==5:
        return RGrad5(X, ranks, A)
    elif len(X.shape)== 6:
        return RGrad6(X, ranks, A)

In [10]:
import time
def PGD(Y, X, trueA, p_vec1, p_vec, mtl_rank, max_iter, tol):
    p1, N = Y.shape
    p, _ = X.shape
    A_inittmp = (Y@X.T)/N
    A_initts = TTOI(A_inittmp.reshape(p_vec1+p_vec,order="F"),mtl_rank[1:-1],2,1e-4)[-1]
    #factors = matrix_product_state(A_inittmp.reshape(p_vec1+p_vec,order="F"), rank=mtl_rank)
    #A_initts = tt_to_tensor(factors)
    Amat = A_initts.reshape(p1,p,order="F")
    chg_list = [None] * max_iter
    est_list = [None] * max_iter
    n = 0
    chg = np.inf
    while n < max_iter and chg > tol:
        start = time.time()
        P = RGrad(Amat.reshape(p_vec1+p_vec,order="F"),mtl_rank,((Amat@X-Y)@X.T)/N)
        Pmat = P.reshape(p1,p,order="F")
        eta = 0.1*N*np.square(np.linalg.norm(Pmat,"fro"))/np.square(np.linalg.norm(Pmat@X,"fro"))
        A_tmpmat = Amat - eta*Pmat
        factors = matrix_product_state(A_tmpmat.reshape(p_vec1+p_vec,order="F"), rank=mtl_rank)
        A_newts = tt_to_tensor(factors)
        A_newmat = A_newts.reshape(p1,p,order="F")
        chg_list[n] = np.linalg.norm(A_newmat-Amat, "fro")
        est_list[n] = np.linalg.norm(A_newmat-trueA, "fro")
        chg = np.linalg.norm(A_newmat-Amat, "fro")
        #print(chg)
        if chg>10:
            break
        n=n+1
        Amat = A_newmat
        

    return ([x for x in chg_list if x is not None],[x for x in est_list if x is not None],trueA,Amat)

Tucker

In [11]:
def ten2mat(X, mode):
    p_vec = list(X.shape)
    p = np.prod(p_vec)
    X1 = np.moveaxis(X, mode, 0)
    return X1.reshape([p_vec[mode], int(p/p_vec[mode])], order="F")


def mat2ten(X, mode, p):
    p.insert(0, p.pop(mode))
    X1 = X.reshape(p,order="F")
    return np.moveaxis(X1, 0, mode)

In [12]:
def RGrad_Tucker3(X, ranks, A):
    p_vec = list(X.shape)
    p1, p2, p3 = p_vec
    r1, r2, r3 = ranks
    svd1 = np.linalg.svd(ten2mat(X, 0), full_matrices=False)
    U1, W1 = svd1[0][:,:r1], svd1[2][:r1,:].T
    svd2 = np.linalg.svd(ten2mat(X, 1), full_matrices=False)
    U2, W2 = svd2[0][:,:r2], svd2[2][:r2,:].T
    svd3 = np.linalg.svd(ten2mat(X, 2), full_matrices=False)
    U3, W3 = svd3[0][:,:r3], svd3[2][:r3,:].T

    delta4 = multi_mode_dot(A, [U1@U1.T, U2@U2.T, U3@U3.T])
    delta1 = mat2ten((np.identity(p1) - U1 @ U1.T) @ ten2mat(A, 0) @ W1 @ W1.T, 0, [p1,p2,p3])
    delta2 = mat2ten((np.identity(p2) - U2 @ U2.T) @ ten2mat(A, 1) @ W2 @ W2.T, 1, [p1,p2,p3])
    delta3 = mat2ten((np.identity(p3) - U3 @ U3.T) @ ten2mat(A, 2) @ W3 @ W3.T, 2, [p1,p2,p3])

    return delta1+delta2+delta3+delta4

In [13]:
def RGrad_Tucker4(X, ranks, A):
    p_vec = list(X.shape)
    p1, p2, p3, p4 = p_vec
    r1, r2, r3, r4 = ranks
    svd1 = np.linalg.svd(ten2mat(X, 0), full_matrices=False)
    U1, W1 = svd1[0][:,:r1], svd1[2][:r1,:].T
    svd2 = np.linalg.svd(ten2mat(X, 1), full_matrices=False)
    U2, W2 = svd2[0][:,:r2], svd2[2][:r2,:].T
    svd3 = np.linalg.svd(ten2mat(X, 2), full_matrices=False)
    U3, W3 = svd3[0][:,:r3], svd3[2][:r3,:].T
    svd4 = np.linalg.svd(ten2mat(X, 3), full_matrices=False)
    U4, W4 = svd4[0][:,:r4], svd4[2][:r4,:].T

    delta5 = multi_mode_dot(A, [U1@U1.T, U2@U2.T, U3@U3.T, U4@U4.T])
    delta1 = mat2ten((np.identity(p1) - U1 @ U1.T) @ ten2mat(A, 0) @ W1 @ W1.T, 0, [p1,p2,p3,p4])
    delta2 = mat2ten((np.identity(p2) - U2 @ U2.T) @ ten2mat(A, 1) @ W2 @ W2.T, 1, [p1,p2,p3,p4])
    delta3 = mat2ten((np.identity(p3) - U3 @ U3.T) @ ten2mat(A, 2) @ W3 @ W3.T, 2, [p1,p2,p3,p4])
    delta4 = mat2ten((np.identity(p4) - U4 @ U4.T) @ ten2mat(A, 3) @ W4 @ W4.T, 3, [p1,p2,p3,p4])

    return delta1+delta2+delta3+delta4+delta5

In [14]:
def RGrad_Tucker5(X, ranks, A):
    p_vec = list(X.shape)
    p1, p2, p3, p4, p5 = p_vec
    r1, r2, r3, r4, r5 = ranks
    svd1 = np.linalg.svd(ten2mat(X, 0), full_matrices=False)
    U1, W1 = svd1[0][:,:r1], svd1[2][:r1,:].T
    svd2 = np.linalg.svd(ten2mat(X, 1), full_matrices=False)
    U2, W2 = svd2[0][:,:r2], svd2[2][:r2,:].T
    svd3 = np.linalg.svd(ten2mat(X, 2), full_matrices=False)
    U3, W3 = svd3[0][:,:r3], svd3[2][:r3,:].T
    svd4 = np.linalg.svd(ten2mat(X, 3), full_matrices=False)
    U4, W4 = svd4[0][:,:r4], svd4[2][:r4,:].T
    svd5 = np.linalg.svd(ten2mat(X, 4), full_matrices=False)
    U5, W5 = svd5[0][:,:r5], svd5[2][:r5,:].T

    delta6 = multi_mode_dot(A, [U1@U1.T, U2@U2.T, U3@U3.T, U4@U4.T, U5@U5.T])
    delta1 = mat2ten((np.identity(p1) - U1 @ U1.T) @ ten2mat(A, 0) @ W1 @ W1.T, 0, [p1,p2,p3,p4,p5])
    delta2 = mat2ten((np.identity(p2) - U2 @ U2.T) @ ten2mat(A, 1) @ W2 @ W2.T, 1, [p1,p2,p3,p4,p5])
    delta3 = mat2ten((np.identity(p3) - U3 @ U3.T) @ ten2mat(A, 2) @ W3 @ W3.T, 2, [p1,p2,p3,p4,p5])
    delta4 = mat2ten((np.identity(p4) - U4 @ U4.T) @ ten2mat(A, 3) @ W4 @ W4.T, 3, [p1,p2,p3,p4,p5])
    delta5 = mat2ten((np.identity(p5) - U5 @ U5.T) @ ten2mat(A, 4) @ W5 @ W5.T, 4, [p1,p2,p3,p4,p5])

    return delta1+delta2+delta3+delta4+delta5+delta6

In [15]:
def RGrad_Tucker6(X, ranks, A):
    p_vec = list(X.shape)
    p1, p2, p3, p4, p5, p6 = p_vec
    r1, r2, r3, r4, r5, r6 = ranks
    svd1 = np.linalg.svd(ten2mat(X, 0), full_matrices=False)
    U1, W1 = svd1[0][:,:r1], svd1[2][:r1,:].T
    svd2 = np.linalg.svd(ten2mat(X, 1), full_matrices=False)
    U2, W2 = svd2[0][:,:r2], svd2[2][:r2,:].T
    svd3 = np.linalg.svd(ten2mat(X, 2), full_matrices=False)
    U3, W3 = svd3[0][:,:r3], svd3[2][:r3,:].T
    svd4 = np.linalg.svd(ten2mat(X, 3), full_matrices=False)
    U4, W4 = svd4[0][:,:r4], svd4[2][:r4,:].T
    svd5 = np.linalg.svd(ten2mat(X, 4), full_matrices=False)
    U5, W5 = svd5[0][:,:r5], svd5[2][:r5,:].T
    svd6 = np.linalg.svd(ten2mat(X, 5), full_matrices=False)
    U6, W6 = svd6[0][:,:r6], svd6[2][:r6,:].T

    delta7 = multi_mode_dot(A, [U1@U1.T, U2@U2.T, U3@U3.T, U4@U4.T, U5@U5.T, U6@U6.T])
    delta1 = mat2ten((np.identity(p1) - U1 @ U1.T) @ ten2mat(A, 0) @ W1 @ W1.T, 0, [p1,p2,p3,p4,p5,p6])
    delta2 = mat2ten((np.identity(p2) - U2 @ U2.T) @ ten2mat(A, 1) @ W2 @ W2.T, 1, [p1,p2,p3,p4,p5,p6])
    delta3 = mat2ten((np.identity(p3) - U3 @ U3.T) @ ten2mat(A, 2) @ W3 @ W3.T, 2, [p1,p2,p3,p4,p5,p6])
    delta4 = mat2ten((np.identity(p4) - U4 @ U4.T) @ ten2mat(A, 3) @ W4 @ W4.T, 3, [p1,p2,p3,p4,p5,p6])
    delta5 = mat2ten((np.identity(p5) - U5 @ U5.T) @ ten2mat(A, 4) @ W5 @ W5.T, 4, [p1,p2,p3,p4,p5,p6])
    delta6 = mat2ten((np.identity(p6) - U6 @ U6.T) @ ten2mat(A, 5) @ W6 @ W6.T, 5, [p1,p2,p3,p4,p5,p6])

    return delta1+delta2+delta3+delta4+delta5+delta6+delta7

In [16]:
def RGrad_Tucker(X, ranks, A):
    if len(X.shape)==3:
        return RGrad_Tucker3(X, ranks, A)
    elif len(X.shape)==4:
        return RGrad_Tucker4(X, ranks, A)
    elif len(X.shape)==5:
        return RGrad_Tucker5(X, ranks, A)
    elif len(X.shape)== 6:
        return RGrad_Tucker6(X, ranks, A)

In [17]:
def HOSVD(A, rank):
    d = len(A.shape)
    factors = []
    for i in range(d):
        tmpU, _, _ = np.linalg.svd(ten2mat(A, i), full_matrices=False)
        factors.append(tmpU[:, :rank[i]])
    return multi_mode_dot(A, [x@x.T for x in factors])


def HOOI(A, rank, iter1):
    return tucker_to_tensor(tucker(A, rank, n_iter_max=iter1))

In [18]:
def PGD_Tucker(Y, X, trueA, p_vec1, p_vec, mtl_rank, max_iter, tol):
    p1, N = Y.shape
    p, _ = X.shape
    A_inittmp = (Y@X.T)/N
    A_initts = HOOI(A_inittmp.reshape(p_vec1+p_vec,order="F"),mtl_rank,2)
    Amat = A_initts.reshape(p1,p,order="F")
    chg_list = [None] * max_iter
    est_list = [None] * max_iter
    n = 0
    chg = np.inf
    while n < max_iter and chg > tol:
        start = time.time()
        P = RGrad_Tucker(Amat.reshape(p_vec1+p_vec,order="F"),mtl_rank,(((Amat@X-Y)@X.T)/N).reshape(p_vec1+p_vec,order="F"))
        Pmat = P.reshape(p1,p,order="F")
        eta = 0.1*N*np.square(np.linalg.norm(Pmat,"fro"))/np.square(np.linalg.norm(Pmat@X,"fro"))
        A_tmpmat = Amat - eta*Pmat
        A_newts = HOSVD(A_tmpmat.reshape(p_vec1+p_vec,order="F"), rank=mtl_rank)
        A_newmat = A_newts.reshape(p1,p,order="F")
        chg_list[n] = np.linalg.norm(A_newmat-Amat, "fro")
        est_list[n] = np.linalg.norm(A_newmat-trueA, "fro")
        chg = np.linalg.norm(A_newmat-Amat, "fro")
        #print(chg)
        if chg>10:
            break
        n=n+1
        Amat = A_newmat
        

    return ([x for x in chg_list if x is not None],[x for x in est_list if x is not None],trueA,Amat)

CP

In [19]:
from tensorly.decomposition import CP
from tensorly.cp_tensor import cp_to_tensor

In [39]:
def PGD_CP(Y, X, trueA, p_vec1, p_vec, mtl_rank, eta, max_iter, tol):
    p1, N = Y.shape
    p, _ = X.shape
    #A_inittmp = (Y@X.T)/N
    #A_initts = cp_to_tensor(CP(mtl_rank,init='random').fit_transform(A_inittmp.reshape(p_vec1+p_vec,order="F")))
    #Amat = A_initts.reshape(p1,p,order="F")
    Amat = trueA
    chg_list = [None] * max_iter
    est_list = [None] * max_iter
    n = 0
    chg = np.inf
    while n < max_iter and chg > tol:
        start = time.time()
        g_k = Amat - (eta*2/(N-1))*(Amat@X-Y)@np.transpose(X)
        g_k = g_k.reshape(p_vec1+p_vec,order="F")
        g_k = cp_to_tensor(CP(mtl_rank).fit_transform(g_k))
        g_k = g_k.reshape(p1,p,order="F")
        chg = np.linalg.norm(g_k-Amat, "fro")
        print(chg)
        chg_list[n] = chg
        est_list[n] = np.linalg.norm(g_k-trueA, "fro")
        print(est_list[n])
        if chg>10:
            break
        Amat = g_k
        n = n+1      

    return ([x for x in chg_list if x is not None],[x for x in est_list if x is not None],trueA,Amat)

Change sample size N

In [ ]:
def d1(ind):
        
    scipy.random.seed() 
    p_vec=[8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 4
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,Y,X,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=300))
        with open('d3R4.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d2(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 4
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,Y,X,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=300))
        with open('d4R4.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d3(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 4
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=300))
        with open('d5R4.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d4(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5,5,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 4
    global U1; global U2; global U3; global core; global U4; global U5; global U6;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    U6 = np.random.normal(loc=0, scale=1, size=p_vec[4]*CP_rank).reshape(p_vec[4],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1),U6[:,0].reshape(p_vec[4],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1),U6[:,i].reshape(p_vec[4],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=300))
        with open('d6R4.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
rr[2][1][-1]

In [ ]:
len(rr[2][0])

In [ ]:
TT1 = []
Tucker1 = []
CP1 = []

In [ ]:
with open('d6R4.pkl','rb') as f:
    result = pickle.load(f)
list1 = []
list2 = []
list3 = []
for i in range(len(result)):
    list1.append(result[i][0][1][-1])
    list2.append(result[i][1][1][-1])
    list3.append(result[i][2][1][-1])
print(np.mean(list1))
print(np.mean(list2))
print(np.mean(list3))
TT1.append(np.mean(list1))
Tucker1.append(np.mean(list2))
CP1.append(np.mean(list3))

In [ ]:
import matplotlib.pyplot as plt
plt.plot([3,4,5,6],TT1)
plt.plot([3,4,5,6],Tucker1)
plt.plot([3,4,5,6],CP1)
plt.xlabel("order")

In [ ]:
with open('d3R3.pkl','rb') as f:
    result = pickle.load(f)
list1 = []
list2 = []
list3 = []
for i in range(len(result)):
    list1.append(result[i][0][1][-1])
    list2.append(result[i][1][1][-1])
    list3.append(result[i][2][1][-1])
print(np.mean(list1))
print(np.mean(list2))
print(np.mean(list3))

Increase rank

In [ ]:
def d1(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 2
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=300))
        with open('1d5R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d2(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 3
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=300))
        with open('1d5R3.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d3(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 4
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=300))
        with open('1d5R4.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
def d4(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    CP_rank = 5
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,CP_rank).reshape([1,1,1,1,1,CP_rank])
    U1 = np.random.normal(loc=0, scale=1, size=p_vec1[0]*CP_rank).reshape(p_vec1[0],CP_rank)
    U2 = np.random.normal(loc=0, scale=1, size=p_vec[0]*CP_rank).reshape(p_vec[0],CP_rank)
    U3 = np.random.normal(loc=0, scale=1, size=p_vec[1]*CP_rank).reshape(p_vec[1],CP_rank)
    U4 = np.random.normal(loc=0, scale=1, size=p_vec[2]*CP_rank).reshape(p_vec[2],CP_rank)
    U5 = np.random.normal(loc=0, scale=1, size=p_vec[3]*CP_rank).reshape(p_vec[3],CP_rank)
    A_ts = tucker_to_tensor((core[:,:,:,:,:,0],[U1[:,0].reshape(p_vec1[0],1),U2[:,0].reshape(p_vec[0],1),U3[:,0].reshape(p_vec[1],1)
                                                ,U4[:,0].reshape(p_vec[2],1),U5[:,0].reshape(p_vec[3],1)]))
    for i in range(1, CP_rank):
        A_ts = A_ts + tucker_to_tensor((core[:,:,:,:,:,i],[U1[:,i].reshape(p_vec1[0],1),U2[:,i].reshape(p_vec[0],1),U3[:,i].reshape(p_vec[1],1)
                                                ,U4[:,i].reshape(p_vec[2],1),U5[:,i].reshape(p_vec[3],1)]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    #print(np.linalg.norm(A_mat,"fro"))
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=CP_rank, eta=0.1, max_iter=2000, tol=1e-3)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[CP_rank]*d, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[CP_rank]*(d-1)+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(30) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=300))
        with open('1d5R5.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

True Tucker

In [41]:
def d1(ind):
        
    scipy.random.seed() 
    p_vec=[8,8]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    Tucker_rank = [2] * d
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,np.prod(Tucker_rank)).reshape(Tucker_rank)
    U1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:Tucker_rank[0]]
    U2 = np.linalg.svd(np.random.normal(0,1,p_vec[0]*p_vec[0]).reshape(p_vec[0],p_vec[0]))[0][:,0:Tucker_rank[1]]
    U3 = np.linalg.svd(np.random.normal(0,1,p_vec[1]*p_vec[1]).reshape(p_vec[1],p_vec[1]))[0][:,0:Tucker_rank[2]]
    #U4 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:Tucker_rank[3]]
    #U5 = np.linalg.svd(np.random.normal(0,1,p_vec[3]*p_vec[3]).reshape(p_vec[3],p_vec[3]))[0][:,0:Tucker_rank[4]]
    A_ts = tucker_to_tensor((core,[U1,U2,U3]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=4, eta=0.01, max_iter=2000, tol=1e-2)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=Tucker_rank, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[2,2]+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [42]:
d1(10)

0.008346037029221728
0.008346037029221728


(([0.0950663787896509,
   0.08636167885876278,
   0.07848862332651202,
   0.0713680724587726,
   0.06492795732165753,
   0.059102677528422715,
   0.05383255843963943,
   0.049063356581213906,
   0.04474580596871709,
   0.04083520069741563,
   0.03729101086727013,
   0.03407652991467863,
   0.031158551915470813,
   0.02850707755030631,
   0.026095047283089822,
   0.02389809996593414,
   0.021894354609447247,
   0.020064212509488224,
   0.018390176384117985,
   0.016856682755158642,
   0.01544994363270231,
   0.014157793749931116,
   0.012969540233046493,
   0.01187581268277197,
   0.010868413088832905,
   0.009940166587167782,
   0.0090847755176784,
   0.008296680260575713,
   0.007570930711130478,
   0.006903071928218443,
   0.006289046555497859,
   0.005725115290217947,
   0.0052077952484847236,
   0.004733814811568662,
   0.004300082619546413,
   0.003903667882210564,
   0.0035417890823612646,
   0.0032118083658953054,
   0.0029112293320438143,
   0.0026376964445193906,
   0.00238899

In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d1,l),total=300))
        with open('1d3R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [44]:
def d2(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    Tucker_rank = [2] * d
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,np.prod(Tucker_rank)).reshape(Tucker_rank)
    U1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:Tucker_rank[0]]
    U2 = np.linalg.svd(np.random.normal(0,1,p_vec[0]*p_vec[0]).reshape(p_vec[0],p_vec[0]))[0][:,0:Tucker_rank[1]]
    U3 = np.linalg.svd(np.random.normal(0,1,p_vec[1]*p_vec[1]).reshape(p_vec[1],p_vec[1]))[0][:,0:Tucker_rank[2]]
    U4 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:Tucker_rank[3]]
    #U5 = np.linalg.svd(np.random.normal(0,1,p_vec[3]*p_vec[3]).reshape(p_vec[3],p_vec[3]))[0][:,0:Tucker_rank[4]]
    A_ts = tucker_to_tensor((core,[U1,U2,U3,U4]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=8, eta=0.01, max_iter=2000, tol=1e-2)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=Tucker_rank, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[2,4,2]+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [45]:
d2(10)

0.01321286337082251
0.01321286337082251
0.013177485192701791
0.025835377043313946
0.013400538593052574
0.03775418435162364
0.013627175408190063
0.0493958840622416
0.013945284868287567
0.060280558882394676
0.013749609355005634
0.07145148633455178
0.014341289275941624
0.08191082083973567
0.014449793277140136
0.09228763569841221
0.014918491422398003
0.10198870176123163
0.016376259791651738
0.11111452448339589
0.01625877099800521
0.12077115896369597
0.018456961840857836
0.12978548616001204
0.01876169503500918
0.13804424788406536
0.019470034874807016
0.14585275813446358
0.019299989033804144
0.15387749672969564
0.023001658797826424
0.16054102564574316
0.022825951703254747
0.1675203989001476
0.0215055200228211
0.17438300093763734
0.02357701308747543
0.18080030384420356
0.022064229088858904
0.18750574308141824
0.02067865151624151
0.1947068206200808
0.02075563440650208
0.20129254811102107
0.025339973477306807
0.2073380254745865
0.030319126438238732
0.21127027002819482
0.025946028344213497
0.217

KeyboardInterrupt: 

In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d2,l),total=300))
        with open('1d4R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [21]:
def d3(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    Tucker_rank = [2] * d
    global U1; global U2; global U3; global core; global U4; global U5;
    global A_ts; global A_mat
    core = np.random.normal(0,1,np.prod(Tucker_rank)).reshape(Tucker_rank)
    U1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:Tucker_rank[0]]
    U2 = np.linalg.svd(np.random.normal(0,1,p_vec[0]*p_vec[0]).reshape(p_vec[0],p_vec[0]))[0][:,0:Tucker_rank[1]]
    U3 = np.linalg.svd(np.random.normal(0,1,p_vec[1]*p_vec[1]).reshape(p_vec[1],p_vec[1]))[0][:,0:Tucker_rank[2]]
    U4 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:Tucker_rank[3]]
    U5 = np.linalg.svd(np.random.normal(0,1,p_vec[3]*p_vec[3]).reshape(p_vec[3],p_vec[3]))[0][:,0:Tucker_rank[4]]
    A_ts = tucker_to_tensor((core,[U1,U2,U3,U4,U5]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=16, eta=0.01, max_iter=1000, tol=1e-2)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=Tucker_rank, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[2,4,4,2]+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts,Y,X)


In [40]:
rr=d3(10)

0.018897850492625073
0.018897850492625073
0.01935121765425962
0.03461300415409709
0.018931532734076842
0.05092524724916709
0.02059950820703875
0.06593495250414219
0.020475810197497085
0.0805791940974289
0.02416157448049096
0.09459150600561926
0.02131991011127724
0.10804062308204151
0.022654227576834225
0.12082049342155339
0.022468046282613965
0.13374396905719171
0.025244813982587287
0.14533673766536484
0.02640611291348771
0.15689524882547137
0.029035463085279208
0.16718503507535115
0.026921325092177796
0.17812136257770253
0.03379811050031646
0.18648596665898756
0.03341435742696362
0.1951648930147778
0.034990284362077996
0.2041192307896725
0.0317852607192354
0.2133820783896416
0.03131873208118781
0.22313618238802363
0.032484637351581094
0.23196263581613963
0.03630393581599282
0.23913401683716723
0.03478505896997647
0.24760948556895163
0.030438060489587106
0.25688509478909727
0.04373005297204079
0.2616757420961089
0.045154411103347024
0.26599108473855704
0.041909617553965633
0.2722259172

KeyboardInterrupt: 

In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d3,l),total=300))
        with open('1d5R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [27]:
def d4(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,5,5,5]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    Tucker_rank = [2] * d
    global U1; global U2; global U3; global core; global U4; global U5; global U6;
    global A_ts; global A_mat
    core = np.random.normal(0,1,np.prod(Tucker_rank)).reshape(Tucker_rank)
    U1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:Tucker_rank[0]]
    U2 = np.linalg.svd(np.random.normal(0,1,p_vec[0]*p_vec[0]).reshape(p_vec[0],p_vec[0]))[0][:,0:Tucker_rank[1]]
    U3 = np.linalg.svd(np.random.normal(0,1,p_vec[1]*p_vec[1]).reshape(p_vec[1],p_vec[1]))[0][:,0:Tucker_rank[2]]
    U4 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:Tucker_rank[3]]
    U5 = np.linalg.svd(np.random.normal(0,1,p_vec[3]*p_vec[3]).reshape(p_vec[3],p_vec[3]))[0][:,0:Tucker_rank[4]]
    U6 = np.linalg.svd(np.random.normal(0,1,p_vec[4]*p_vec[4]).reshape(p_vec[4],p_vec[4]))[0][:,0:Tucker_rank[5]]
    A_ts = tucker_to_tensor((core,[U1,U2,U3,U4,U5,U6]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()

    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=32, eta=0.01, max_iter=2000, tol=1e-2)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=Tucker_rank, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[2,4,8,4,2]+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


In [34]:
d4(10)

0.5684263887899046
1.47891379195915
0.44924783409754393
0.9946429041782248
0.36661132721296114
1.1302267365759953
0.2985902428156607
0.2819150326876629


KeyboardInterrupt: 

In [ ]:
import pickle
import time
import tqdm
start = time.time()
if __name__ == '__main__':
    l = list(range(300))
    with Pool(35) as p:
        result = list(tqdm.tqdm(p.imap(d4,l),total=300))
        with open('1d6R2.pkl','wb') as f:
            pickle.dump(result, f)

end = time.time()
print(end-start)

In [ ]:
result=d3(10)

In [ ]:
CP converge very slow iter_num lower

In [ ]:
print(result[0][1][-1])
print(result[1][1][-1])
print(result[2][1][-1])
result[0][0]

In [ ]:
print(result[0][1][-1])
print(result[1][1][-1])
print(result[2][1][-1])

In [ ]:
result1[0][1][-1]

In [ ]:
result[0][1][-1]

In [ ]:
np.min(result[0][1])

In [ ]:
def d3(ind):
        
    scipy.random.seed() 
    p_vec=[8,8,6,6,6]
    p = np.prod(p_vec)
    p_vec1=[8]
    d = len(p_vec) + len(p_vec1)
    p1 = np.prod(p_vec1)
    N=500
    Tucker_rank = [3] * d
    global U1; global U2; global U3; global core; global U4; global U5; global U6;
    global A_ts; global A_mat
    core = np.random.normal(0,1,np.prod(Tucker_rank)).reshape(Tucker_rank)
    U1 = np.linalg.svd(np.random.normal(0,1,p_vec1[0]*p_vec1[0]).reshape(p_vec1[0],p_vec1[0]))[0][:,0:Tucker_rank[0]]
    U2 = np.linalg.svd(np.random.normal(0,1,p_vec[0]*p_vec[0]).reshape(p_vec[0],p_vec[0]))[0][:,0:Tucker_rank[1]]
    U3 = np.linalg.svd(np.random.normal(0,1,p_vec[1]*p_vec[1]).reshape(p_vec[1],p_vec[1]))[0][:,0:Tucker_rank[2]]
    U4 = np.linalg.svd(np.random.normal(0,1,p_vec[2]*p_vec[2]).reshape(p_vec[2],p_vec[2]))[0][:,0:Tucker_rank[3]]
    U5 = np.linalg.svd(np.random.normal(0,1,p_vec[3]*p_vec[3]).reshape(p_vec[3],p_vec[3]))[0][:,0:Tucker_rank[4]]
    U6 = np.linalg.svd(np.random.normal(0,1,p_vec[4]*p_vec[4]).reshape(p_vec[4],p_vec[4]))[0][:,0:Tucker_rank[5]]
    A_ts = tucker_to_tensor((core,[U1,U2,U3,U4,U5,U6]))
    A_ts = A_ts * 5/np.linalg.norm(A_ts.reshape(p1,p,order="F"),"fro")
    A_mat = A_ts.reshape(p1,p,order="F")
    global X; global Y
    #X = torch.empty((p, N), dtype=torch.float64)
    X = torch.tensor(np.random.normal(0,1,p*N).reshape(p,N))
    Y = torch.empty((p1, N), dtype=torch.float64)
    for s in range(N):
        #X[:,s] = torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p),covariance_matrix=torch.eye(p)).sample()
        Y[:,s] = torch.tensor(A_mat) @ X[:,s] + torch.distributions.multivariate_normal.MultivariateNormal(loc=torch.zeros(p1),covariance_matrix=torch.eye(p1)).sample()
    X = X.numpy()
    Y = Y.numpy()
    print("Hello")
    result_CP = PGD_CP(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=100, eta=0.01, max_iter=200, tol=5e-2)
    result_Tucker = PGD_Tucker(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=Tucker_rank, max_iter=2000, tol=1e-3)
    result_TT = PGD(Y, X, trueA=A_mat, p_vec1=p_vec1, p_vec=p_vec, mtl_rank=[1]+[3,9,27,9,3]+[1], max_iter=2000, tol=1e-3)
    return (result_TT, result_Tucker,result_CP,A_ts)


result1=d3(10)

In [ ]:
result1=d3(10)

In [ ]:
print(np.min(result[0][1]))
print(result[1][1][-1])
print(result[2][1][-1])

In [ ]:
result[0][1]